<h2 id="Contents">Contents<a href="#Contents"></a></h2>
        <ol>
        <ol><li><a class="" href="#Basic-inheritance">Basic inheritance</a></li>
<ol><li><a class="" href="#Extending-built-ins">Extending built-ins</a></li>
<li><a class="" href="#Overriding-and-super">Overriding and super</a></li>
</ol><li><a class="" href="#Multiple-inheritance">Multiple inheritance</a></li>
<ol><li><a class="" href="#The-Diamond-Problem">The Diamond Problem</a></li>
<li><a class="" href="#Different-sets-of-arguments">Different sets of arguments</a></li>
</ol><li><a class="" href="#Polymorphism">Polymorphism</a></li>
<li><a class="" href="#Duck-Typing">Duck Typing</a></li>
<li><a class="" href="#Abstract-Base-Classes">Abstract Base Classes</a></li>
<ol><li><a class="" href="#Creating-an-abstract-base-class">Creating an abstract base class</a></li>
</ol><li><a class="" href="#Case-study">Case study</a></li>
</ol>

>In the programming world, duplicate code is considered evil.

There are many ways to merge pieces of code or objects that have a similar
functionality. like:
* Basic inheritance
* Inheriting from built-ins
* Multiple inheritance
* Polymorphism and duck typing

## Basic inheritance
Technically, every class we create uses inheritance. All Python classes are subclasses
of the special class named `object`.

In [1]:
class MyClass(object):
    pass

The class `MyClass` inherits from the `object` class.<br>
A *superclass*, or *parent class*, is a class that is being inherited from. A *subclass* is a class that is inheriting from a superclass.
>The simplest and most obvious use of
inheritance is to add functionality to an existing class.

In [2]:
class Contact:
    all_contacts = []
    def __init__(self, name, email):
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)

The `all_contacts` list, because it is part of the class definition, is shared by all instances of this class. To show some more benefits of inheritance, let's create a new Supplier class that acts like our Contact class,
but has an additional order method:


In [3]:
class Supplier(Contact):
    def order(self, order):
        print("If this were a real system we would send "
        "'{}' order to '{}'".format(order, self.name))

Now, if we test this class in our trusty interpreter, we see that all `contacts`, including
`suppliers`, accept a name and e-mail address in their `__init__`, but only `suppliers`
have a functional `order` method. Thus we can avoid giving some useless methods to other subclasses.

### Extending built-ins
One interesting use of this kind of inheritance is adding functionality to built-in
classes. In the Contact class seen earlier, we are adding contacts to a list of all
contacts. What if we also wanted to search that list by name? Well, we could add
a method on the `Contact` class to search it, but it feels like this method actually
belongs to the `list` itself. We can do this using inheritance:

In [26]:
class ContactList(list):

    def search(self, name):
        '''Return all contacts that contain the search value
        in their name.'''
        matching_contacts = []
        for contact in self:
            if name in contact.name:
                matching_contacts.append(contact)
        return matching_contacts


class Contact:
    all_contacts = ContactList()
    
    def __init__(self, name, email):
        self.name = name
        self.email = email
        self.all_contacts.append(self)

c1 = Contact("John A", "johna@example.net")
c2 = Contact("John B", "johnb@example.net")
c3 = Contact("Jenna C", "jennac@example.net")
c4 = Contact("John D", "johnd@example.net")
[c.name for c in Contact.all_contacts.search('John')]

['John A', 'John B', 'John D']

Instead of instantiating a normal `list` as our class variable, we create a new
`ContactList` class that extends the built-in list. Now `list` has a method `search`.

>In reality, the `[]` syntax is actually so-called syntax sugar that calls the `list()`
constructor under the hood. 

In [27]:
isinstance([], object), [] == list()

(True, True)

### Overriding and super
Any method in a subclass can override the method in the superclass.

In [28]:
class Friend(Contact):
    
    def __init__(self, name, email, phone):
        self.name = name
        self.email = email
        self.phone = phone

Here the `__init__` method in the `Contact` class is overridden to accept a `phone` attribute. <br>
What we really need is a way to execute the original `__init__` method on the
`Contact` class. This is what the `super` function does; it returns the object as an
instance of the parent class, allowing us to call the parent method directly:


In [29]:
class Friend(Contact):
    def __init__(self, name, email, phone):
        super().__init__(name, email)
        self.phone = phone

A `super()` call can be made inside any method, not just `__init__`. This means all
methods can be modified via overriding and calls to super. The call to super can
also be made at any point in the method; we don't have to make the call as the first
line in the method. For example, we may need to manipulate or validate incoming
parameters before forwarding them to the superclass.

## Multiple inheritance
Multiple inheritance is a touchy subject. In principle, it's very simple: a subclass that
inherits from more than one parent class is able to access functionality from both of
them. In practice, this is less useful than it sounds and many expert programmers
recommend against using it.
>As a rule of thumb, if you think you need multiple inheritance, you're
probably wrong, but if you know you need it, you're probably right.


The simplest and most useful form of multiple inheritance is called a mixin. A mixin
is generally a superclass that is not meant to exist on its own, but is meant to be
inherited by some other class to provide extra functionality. <br>
For example, let's say
we wanted to add functionality to our Contact class that allows sending an e-mail
to self.email. Sending e-mail is a common task that we might want to use on many
other classes. So, we create a new class called `MailSender` and use it as a parent class.

In [ ]:
class MailSender:
    def send_mail(self, message):
        print("Sending mail to " + self.email)
        # Add e-mail logic here

        
class EmailableContact(Contact, MailSender):
    pass

However, we can do the same thing without use of double inheritance. For example:
* We could have used single inheritance and added the `send_mail` function
to the subclass. The disadvantage here is that the e-mail functionality then
has to be duplicated for any other classes that need e-mail.
* We can create a standalone Python function for sending an e-mail, and just
call that function with the correct e-mail address supplied as a parameter
when the e-mail needs to be sent.
* We could have explored a few ways of using composition instead of
inheritance. For example, `EmailableContact` could have a `MailSender`
object instead of inheriting from it.
* We could *monkey-patch* the `Contact` class to have a `send_mail` method
after the class has been created. This is done by defining a function that accepts
the self argument, and setting it as an attribute on an existing class.


### The Diamond Problem
We can use multiple inheritance to add this new class as a parent of our existing
`Friend` class. The tricky part is that we now have two parent `__init__ `methods
both of which need to be initialized. And they need to be initialized with different
arguments. We could start with a naive approach:

In [30]:
class AddressHolder:
    def __init__(self, street, city, state, code):
        self.street = street
        self.city = city
        self.state = state
        self.code = code

class Friend(Contact, AddressHolder):
    def __init__(
    self, name, email, phone,street, city, state, code):
        Contact.__init__(self, name, email)
        AddressHolder.__init__(self, street, city, state, code)
        self.phone = phone

<img src="img/03_01.png">

Using the above approach, we'll see that the parent class has been set up twice. With
the `object` class, that's relatively harmless, but in some situations, it could spell
disaster. *Imagine trying to connect to a database twice for every request!* <br>
**This is the diamond problem.**
<br><hr>
<img src="img/03_02.png">

In [31]:
class BaseClass:
    num_base_calls = 0
    def call_me(self):
        print("Calling method on Base Class")
        self.num_base_calls += 1


class LeftSubclass(BaseClass):
    num_left_calls = 0
    def call_me(self):
        BaseClass.call_me(self)
        print("Calling method on Left Subclass")
        self.num_left_calls += 1


class RightSubclass(BaseClass):
    num_right_calls = 0
    def call_me(self):
        BaseClass.call_me(self)
        print("Calling method on Right Subclass")
        self.num_right_calls += 1

        
class Subclass(LeftSubclass, RightSubclass):
    num_sub_calls = 0
    def call_me(self):
        LeftSubclass.call_me(self)
        RightSubclass.call_me(self)
        print("Calling method on Subclass")
        self.num_sub_calls += 1

In [32]:
s = Subclass()
s.call_me()

Calling method on Base Class
Calling method on Left Subclass
Calling method on Base Class
Calling method on Right Subclass
Calling method on Subclass


In [33]:
print(s.num_sub_calls,s.num_left_calls,s.num_right_calls,s.num_base_calls)

1 1 1 2


**Thus we can clearly see the base class's call_me method being called twice.**<br>
The thing to keep in mind with multiple inheritance is that we only want to call
the *"next"* method in the class hierarchy, not the *"parent"* method. In fact, that next
method may not be on a parent or ancestor of the current class. The `super` keyword
comes to our rescue once again. Indeed, `super` was originally developed to make
complicated forms of multiple inheritance possible. 

In [34]:
class BaseClass:
    num_base_calls = 0
    def call_me(self):
        print("Calling method on Base Class")
        self.num_base_calls += 1


class LeftSubclass(BaseClass):
    num_left_calls = 0
    def call_me(self):
        super().call_me()
        print("Calling method on Left Subclass")
        self.num_left_calls += 1


class RightSubclass(BaseClass):
    num_right_calls = 0
    def call_me(self):
        super().call_me()
        print("Calling method on Right Subclass")
        self.num_right_calls += 1


class Subclass(LeftSubclass, RightSubclass):
    num_sub_calls = 0
    def call_me(self):
        super().call_me()
        print("Calling method on Subclass")
        self.num_sub_calls += 1


s = Subclass()
s.call_me()

Calling method on Base Class
Calling method on Right Subclass
Calling method on Left Subclass
Calling method on Subclass


In [35]:
print(s.num_sub_calls,s.num_left_calls,s.num_right_calls,s.num_base_calls)

1 1 1 1


The `supe`r call is not calling the method on the
superclass of `LeftSubclass` (which is `BaseClass`). Rather, it is calling `RightSubclass`,
even though it is not a direct parent of `LeftSubclass`! This is the *next* method, not
the *parent* method. `RightSubclass` then calls `BaseClass` and the `super` calls have
ensured each method in the class hierarchy is executed once.

### Different sets of arguments

How can we manage different sets of arguments when using super? We don't
necessarily know which class `super` is going to try to initialize first. Even if we
did, we need a way to pass the "extra" arguments so that subsequent calls to
`super`, on other subclasses, receive the right arguments.<br>
Specifically, if the first call to super passes the name and email arguments
to `Contact.__init__`, and `Contact.__init__` then calls super, it needs to
be able to pass the address-related arguments to the "next" method, which is
`AddressHolder.__init__`.

The only way to solve this problem is to plan for it from the beginning. We
have to design our base class parameter lists to accept **keyword arguments** for any
parameters that are not required by every subclass implementation. Finally, we must
ensure the method freely accepts unexpected arguments and passes them on to its
super call, in case they are necessary to later methods in the inheritance order.

In [36]:
class Contact:
    all_contacts = []
    def __init__(self, name='', email='', **kwargs):
        super().__init__(**kwargs)
        self.name = name
        self.email = email
        self.all_contacts.append(self)


class AddressHolder:
    def __init__(self, street='', city='', state='', code='',
    **kwargs):
        super().__init__(**kwargs)
        self.street = street
        self.city = city
        self.state = state
        self.code = code


class Friend(Contact, AddressHolder):
    def __init__(self, phone='', **kwargs):
        super().__init__(**kwargs)
        self.phone = phone

We've changed all arguments to keyword arguments by giving them an empty
string as a default value. We've also ensured that a `**kwargs` parameter is included
to capture any additional parameters that our particular method doesn't know what
to do with. It passes these parameters up to the next class with the super call.

The previous example does what it is supposed to do. But it's starting to look messy,
and it has become difficult to answer the question, What arguments do we need to pass
into `Friend.__init__`? This is the foremost question for anyone planning to use the
class, so a docstring should be added to the method to explain what is happening.<br>
Further, even this implementation is insufficient if we want to reuse variables in
parent classes. When we pass the `**kwargs` variable to super, the dictionary does
not include any of the variables that were included as explicit keyword arguments.
For example, in `Friend.__init__`, the call to super does not have phone in the
kwargs dictionary. If any of the other classes need the phone parameter, we need
to ensure it is in the dictionary that is passed. Worse, if we forget to do this, it will
be tough to debug because the superclass will not complain, but will simply assign
the default value (in this case, an empty string) to the variable.

## Polymorphism

Polymorphism is a fancy name describing a simple concept: different behaviors happen depending on
which subclass is being used, without having to explicitly know what the subclass
actually is. As an example, imagine a program that plays audio files.

We can use inheritance with polymorphism to simplify the design. Each type of
file can be represented by a different subclass of `AudioFile`, for example, `WavFile`,
`MP3File`. Each of these would have a `play()` method, but that method would be
implemented differently for each file to ensure the correct extraction procedure is
followed. The media player object would never need to know which subclass of
`AudioFile` it is referring to; it just calls `play()` and polymorphically lets the object
take care of the actual details of playing.

In [37]:
class AudioFile:
    def __init__(self, filename):
        if not filename.endswith(self.ext):
            raise Exception("Invalid file format")
        self.filename = filename


class MP3File(AudioFile):
    ext = "mp3"
    def play(self):
        print("playing {} as mp3".format(self.filename))


class WavFile(AudioFile):
    ext = "wav"
    def play(self):
        print("playing {} as wav".format(self.filename))


class OggFile(AudioFile):
    ext = "ogg"
    def play(self):
        print("playing {} as ogg".format(self.filename))

All audio files check to ensure that a valid extension was given upon initialization.
We notice that the `__init__` method in the parent class is able to access
the ext class variable from different subclasses? That's polymorphism at work.

In [39]:
ogg = OggFile("myfile.ogg")
ogg.play()

playing myfile.ogg as ogg


In [40]:
mp3 = MP3File("myfile.mp3")
mp3.play()

playing myfile.mp3 as mp3


In [41]:
not_an_mp3 = MP3File("myfile.ogg")

Exception: Invalid file format

## Duck Typing

Duck typing in Python allows us to use any object that provides the required behavior without forcing it to be a subclass. The dynamic nature of Python makes this trivial. 
>“If it looks like a duck and quacks like a duck, it’s a duck” <br>

In [46]:
class Specialstring:
    def __len__(self):
        return 21
  
# Driver's code
if __name__ == "__main__":
  
    string = Specialstring()
    print(len(string))

21


The following example does not extend AudioFile, but it can be interacted with in
Python using the exact same interface.

In [44]:
class FlacFile:
    def __init__(self, filename):
        if not filename.endswith(".flac"):
            raise Exception("Invalid file format")
        self.filename = filename

    def play(self):
        print("playing {} as flac".format(self.filename))

f = FlacFile("myfile.flac")
f.play()

playing myfile.flac as flac


Of course, just because an object satisfies a particular interface (by providing required
methods or attributes) does not mean it will simply work in all situations. It has to
fulfill that interface in a way that makes sense in the overall system. Just because an
object provides a `play()` method does not mean it will automatically work with a
media player.

## Abstract Base Classes

While duck typing is useful, it is not always easy to tell in advance if a class is going
to fulfill the protocol you require. Therefore, Python introduced the idea of abstract
base classes. **Abstract base classes**, or **ABCs**, define a set of methods and properties
that a class must implement in order to be considered a duck-type instance of that
class. The class can extend the abstract base class itself in order to be used as an
instance of that class, but it must supply all the appropriate methods.

In [48]:
from collections.abc import Container

Container.__abstractmethods__

frozenset({'__contains__'})

So, the `Container` class has exactly one abstract method that needs to be
implemented, `__contains__`. Let's see how this works.

In [50]:
class OddContainer:
    def __contains__(self, x):
        if not isinstance(x, int) or not x % 2:
            return False
        return True

odd_container = OddContainer()
isinstance(odd_container, Container), issubclass(OddContainer, Container)

(True, True)

We see that `OddContainer` is a subclass and instance of `Container`.

And that is why duck typing is way more awesome than classical polymorphism.
We can create *is a* relationships without the overhead of using inheritance (or
worse, multiple inheritance).<br>
The interesting thing about the `Container` `ABC` is that any class that implements
it gets to use the `in` keyword for free. In fact, in is just syntax sugar that delegates
to the `__contains__` method. Any class that has a `__contains__` method is a
`Container` and can therefore be queried by the in keyword, for example:

In [52]:
1 in odd_container, 2 in odd_container,  "a string" in odd_container

(True, False, False)

### Creating an abstract base class

The abc module provides the tools you need to create an abstract base class, however, this requires some of Python's most arcane concepts:

In [54]:
import abc
class MediaLoader(metaclass=abc.ABCMeta):

    @abc.abstractmethod
    def play(self):
        pass

    @abc.abstractproperty
    def ext(self):
        pass
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is MediaLoader:
            attrs = set(dir(C))
            if set(cls.__abstractmethods__) <= attrs:
                return True
        return NotImplemented

We see the `@abc.abstractmethod` and `@abc.abstractproperty` constructs.
These are Python decorators which insures that any subclass of this class must
implement that method or supply that property in order to be considered a
proper member of the class.

In [56]:
class Wav(MediaLoader):
    pass

x = Wav()

TypeError: Can't instantiate abstract class Wav with abstract methods ext, play

In [57]:
class Ogg(MediaLoader):
    ext = '.ogg'
    def play(self):
        pass

o = Ogg()

In [62]:
isinstance(o, MediaLoader), issubclass(Ogg, MediaLoader)

(True, True)

Going back to the `MediaLoader` ABC, let's dissect that `__subclasshook__` method.
It is basically saying that any class that supplies concrete implementations of all the
abstract attributes of this ABC should be considered a subclass of `MediaLoader`,
even if it doesn't actually inherit from the `MediaLoader` class.

In [64]:
class Ogg():
    ext = '.ogg'
    def play(self):
        print("this will play an ogg file")

isinstance(Ogg(), MediaLoader), issubclass(Ogg, MediaLoader)

(True, True)

In [65]:
class Ogg():
    ext = '.ogg'
    def run(self):
        print("this will run an ogg file")

isinstance(Ogg(), MediaLoader), issubclass(Ogg, MediaLoader)

(False, False)

## Case study

We'll be designing a simple real estate application that allows an *agent* to manage *properties*
available for *purchase* or *rent*. There will be two types of properties: *apartments* and
*houses*. The *agent* needs to be able to enter a few relevant details about new properties,
list all currently available properties, and mark a property as *sold* or *rented*.

<img src = "img/03_03.png" align='middle' style="width:80%">

**Static methods** are associated only with a class (something like class variables), rather than a specific object
instance. Hence, they have no self argument. Because of this, the `super` keyword
won't work (there is no parent object, only a parent class), so we simply call the static
method on the parent class directly. This method uses the Python dict constructor
to create a dictionary of values that can be passed into `__init__`. The value for each
key is prompted with a call to input.